In [6]:
from datasets import load_dataset

squad_balanced = load_dataset("csv", data_files="../../data/compare_baseline_balanced_4b.csv", split='train')
print(squad_balanced)

Dataset({
    features: ['source', 'target', 'dataset', 'category', 'squad_question4b', 'balanced_question4b'],
    num_rows: 1346
})


In [9]:
def filter_to_retain_na(data):
    return data["category"] == "NA"

def categorise_dataset(data):
    target = data["target"].lower()
    if any(word in target for word in ["what"]):
        data["category"] = "description"
    elif any(
        word in target
        for word in [
            "how did",
            "how does",
            "how do",
            "compute",
            "calculate",
            "how can",
            "how should",
            "how would",
            "how will",
            "how to",
        ]
    ):
        data["category"] = "method"
    elif any(
        word in target
        for word in [
            "where",
            "when",
            "who",
            "how",
            "which",
        ]
    ):
        data["category"] = "recall"
    elif any(word in target for word in ["why"]):
        data["category"] = "explanation"
    else:
        data["category"] = "NA"

    return data

balanced_dataset_filtered_na = (
  squad_balanced
  .select_columns(['balanced_question4b'])
  .rename_column("balanced_question4b", "target")
  .add_column("category", ["NA"] * len(squad_balanced))
  .map(categorise_dataset)
  .filter(filter_to_retain_na)
)

balanced_dataset_filtered_na.to_csv("../../data/balanced_dataset_na.csv")


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

3362